In [4]:
#####################################
### Carregar arquivos no supabase ###
#####################################

In [5]:
# imports
from supabase import create_client, Client
import pandas as pd
import os

In [6]:
# Conexão com Supabase
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [7]:
# Carregar dados tratados
df_to_load = pd.read_csv("../data/processed/vendas_processed/base_vendas_tratada.csv")

In [8]:
res = supabase.table("vendas").select("*").execute()
df_supabase_atual = pd.DataFrame(res.data)

In [9]:
chave = ["Data da Venda", "Produto", "Cliente", "Total (R$)"]

In [10]:
df_merge = df_to_load.merge(df_supabase_atual[chave], on=chave, how="left", indicator=True)
df_para_inserir = df_merge[df_merge["_merge"] == "left_only"].drop(columns=["_merge"])


In [11]:
print(f"📥 Linhas novas detectadas: {len(df_para_inserir)}")


📥 Linhas novas detectadas: 0


In [12]:
# Inserir no Supabase (Em lote)

if not df_para_inserir.empty:
    df_para_inserir = df_para_inserir.fillna(None)
    dados = df_para_inserir.to_dict(orient="records")
    supabase.table("vendas").insert(dados).execute()
    print("✅ Novos dados inseridos com sucesso!")
else:
    print("✅ Nenhuma nova linha para inserir.")

✅ Nenhuma nova linha para inserir.


In [13]:
# # pegar caminho atual
caminho_raiz = '/home/lucas/Documentos/empresa-lucas-dashboard'

# # sincronizar para o arquivo .py
!jupytext --to py $caminho_raiz/notebooks/load_vendas.ipynb -o $caminho_raiz/etl/load/load_vendas.py

[jupytext] Reading /home/lucas/Documentos/empresa-lucas-dashboard/notebooks/load_vendas.ipynb in format ipynb
[jupytext] Writing /home/lucas/Documentos/empresa-lucas-dashboard/etl/load/load_vendas.py (destination file replaced)
